# Web scraping Wikipedia List of postal codes of Canada-Toronto
## Project part 1 - Coursera
### By Isaac Mendoza

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Wikipedia List of postal codes to Dataframe pandas

In [43]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(URL).text
soup=BeautifulSoup(source, 'lxml')
table=soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [44]:
df_pro = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df_pro.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [45]:
df_pro=df_pro.groupby(['Postcode'], as_index=False, sort=False).agg(lambda x:','.join(set(x)))
df_pro.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Not assigned


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [49]:
df_pro.loc[df_pro['Neighbourhood']=='Not assigned','Neighbourhood']= \
df_pro.loc[df_pro['Neighbourhood']=='Not assigned','Borough']
#So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df_pro.loc[df_pro['Postcode'].str.contains(r'(M9)'),['Neighbourhood','Borough']]= "Queen's Park"
df_pro.shape

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


(103, 3)